In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import csv
import time
import os
import numpy as np
import concurrent.futures
import random

In [2]:
df = pd.read_csv('initial_test_companies_url.csv', header=None)
df.columns = ['URL', 'Category']

#### Remove duplicate values

In [3]:
df.drop_duplicates(subset='URL', keep=False, inplace=True)
df = df.reset_index(drop=True)

In [59]:
elements = len(df)
n = 2
while elements%n != 0:
    elements -=1

split_step = int(elements/n)
split_from = 0
split_to = split_step
links = []
for i in range(n-1):
    links.append(df.iloc[split_from:split_to,0])
    split_from = split_to
    split_to = split_to + split_step
    
links.append(df.iloc[split_from:,0])


#### Data Scraping

In [5]:
file_name = 'company_data.csv'
data_row = {}

fieldnames = ['Company Name', 'Registration code', 'VAT', 
            'Manager', 'Address', 'Website', 'Work hours', 
            'SS insurer code', 'Transport', 'Authorized capital', 
            'Company age', 'Employees', 'Average salary', 
            'Social insurance taxes', 'Sales revenue From', 
            'Sales revenue To', 'Rating', 'No. Rated', 'Category',
            'Other Categories', 'Facebook']

In [6]:
data_row.keys()

dict_keys([])

In [78]:
def getProxyList(v=True):
    """Scrapes and return a list of available free proxies"""
    URL = 'https://free-proxy-list.net/'
    html = requests.get(URL, timeout=3)
    soup = BeautifulSoup(html.content, 'lxml')

    # Find the first tbody tag and retrieve al tr with data td stored in list
    table = soup.find('tbody')
    rows = [row.find_all('td') for row in table.find_all('tr')]


    # Append a proxy adress to the list if anonimity is 'elite proxy'
    proxy_list = []
    for row in rows:
        if row[4].text == 'elite proxy' and row[6].text == 'yes':
            proxy_list.append(row[0].text + ':' + row[1].text)
    if v == True:
        print(f'{len(proxy_list)} Have been found.')
    return proxy_list

In [79]:
def loadPage(URL, delay, headers, v=True):
    """Requests a URL with with random header and available proxy"""
    proxy_list = getProxyList()
    random.shuffle(proxy_list)
    
    header = {'User-Agent': random.choice(headers)}
    
    for proxy in proxy_list: 
        try:
            if v == True:
                print('Trying proxy: {}'.format(proxy))
            r = requests.get(URL, proxies={'https': proxy}, headers=header, timeout=5)
            break
        except:
            if v == True:
                print('{} is not valid proxy.'.format(proxy))
            continue
    if v == True:
        print(f'Succes! Using proxy: {proxy}')   
    soup = BeautifulSoup(r.content, 'lxml')
    time.sleep(delay)
    return (soup, proxy)

In [80]:
def unusuaTrafficCheck(soup):
    """Checks for capcha"""
    result = soup.find('div',{'class':'content unusualTraffic'})
    return bool(result)

In [81]:
def getHeaderName(ind, table_rows):
    return table_rows[ind].find_all('td')[1].get_text()

def getTableData(ind, table_rows):
    return table_rows[ind].find_all('td')[2].get_text().strip()

In [82]:
def parseTableData(soup):
    """Parse company data table"""

    table_rows = soup.find('table').find_all('tr')
    
    # Data headers of intrest
    data_filter = ['Registration code', 'VAT', 'Manager', 'Address',
                  'Website', 'Work hours', 'SS insurer code',
                  'Average salary', 'Social insurance taxes',
                  'Transport', 'Authorized capital', 'Company age', 'Facebook']

    for row in table_rows:
        header_name = getHeaderName(table_rows.index(row), table_rows)
        table_data = getTableData(table_rows.index(row), table_rows)
        if header_name in data_filter:
            data_row[header_name] = table_data
        elif header_name == 'Employees':
            data_row[header_name] = table_data.split(' ')[0]
        elif header_name == 'Sales revenue':
            result = re.search(r'\d+\s*\d*\s*[-]\s*\d*\s*\d*', table_data)
            data_row[header_name + ' From'] = result[0].replace(' ', '').split('-')[0]
            data_row[header_name + ' To'] = result[0].replace(' ', '').split('-')[1]
        elif header_name == 'Rating':
            try:
                rating = table_rows[table_rows.index(row)].find_all('td')[2].select('span')[0].get_text()
                no_rated = table_rows[table_rows.index(row)].find_all('td')[2].select('span')[2].get_text()
                data_row[header_name] = rating
                data_row['No. Rated'] = no_rated
            except IndexError:
                continue

In [83]:
def getCompanyName(soup):
    """Adds Company Name to the data row"""
    data_row['Company Name'] = soup.select('.fn')[0].get_text()

def parseCategories(soup):
    # Get main category and subcategories
    try:
        category = soup.find('div', {'class':'floatLeft about'}).find_all('a')
    except AttributeError:
        category = soup.find('div', {'class':'about floatLeft'}).find_all('a')
    else:
        data_row['Category'] = category[0]['title']
        # Run this part only if there is more than 1 category.
        other_categories = [i['title'] for i in category if len(category)>1]
        data_row['Other Categories'] = other_categories

In [84]:
def writeHeaders(file_name, fieldnames):
    with open(file_name, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

def writeDataRow(file_name, data_row, fieldnames):
    with open(file_name, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow(data_row)

In [70]:
def loadUserAgents(filename):
    '''Loads User-Agents from csv file'''
    with open(filename, 'r', newline='') as f:
        reader = csv.reader(f)
        user_agents = []
        for row in reader:
            user_agents.append(row[0])
    return user_agents

user_agents = loadUserAgents('../scraping_tools/user_agents.csv')

def scrape(links):
    for i in links: 
        start = time.perf_counter()
        soup, proxy = loadPage(i, 1, user_agents)

        if unusuaTrafficCheck(soup):
            print('Capcha detected!')

        getCompanyName(soup)
        parseTableData(soup)
        parseCategories(soup)
        writeDataRow(file_name, data_row, fieldnames)
        finish = time.perf_counter()
        print(f'Succesfully scraped URL: {i}')
        print(f'Scraping took: {finish-start} seconds using proxy: {proxy}')

In [86]:
writeHeaders(file_name, fieldnames)
with concurrent.futures.ThreadPoolExecutor() as executor:
    task1 = executor.submit(scrape, links[0])
    task2 = executor.submit(scrape, links[1])

        


52 Have been found.
Trying proxy: 43.245.218.217:25
52 Have been found.
Trying proxy: 103.111.225.17:8080
43.245.218.217:25 is not valid proxy.
Trying proxy: 103.240.77.98:30093
103.111.225.17:8080 is not valid proxy.
Trying proxy: 139.59.60.174:8080
139.59.60.174:8080 is not valid proxy.
Trying proxy: 103.110.104.2:8080
103.110.104.2:8080 is not valid proxy.
Trying proxy: 193.239.86.249:3128
193.239.86.249:3128 is not valid proxy.
Trying proxy: 37.120.239.150:3128
37.120.239.150:3128 is not valid proxy.
Trying proxy: 157.230.103.189:36366
103.240.77.98:30093 is not valid proxy.
Trying proxy: 167.71.239.213:8080
157.230.103.189:36366 is not valid proxy.
Trying proxy: 51.254.237.77:3129
Succes! Using proxy: 167.71.239.213:8080
51.254.237.77:3129 is not valid proxy.
Trying proxy: 51.222.173.56:3128
51.222.173.56:3128 is not valid proxy.
Trying proxy: 185.128.136.231:3128
Succes! Using proxy: 185.128.136.231:3128


In [56]:
for i in links[1]:
    print(i)

https://rekvizitai.vz.lt/en/company/erudito_licejus/
https://rekvizitai.vz.lt/en/company/ruklos_jono_stanislausko_vidurine_mokykla/
https://rekvizitai.vz.lt/en/company/santaros_gimnazija/
https://rekvizitai.vz.lt/en/company/kauno_1_oji_specialioji_mokykla/
https://rekvizitai.vz.lt/en/company/kulautuvos_vidurine_mokykla/
https://rekvizitai.vz.lt/en/company/roku_vidurine_mokykla/
https://rekvizitai.vz.lt/en/company/marinos_mizigurskajos_vidurine_mokykla/
https://rekvizitai.vz.lt/en/company/raseiniu_viktoro_petkaus_pagrindine_mokykla/
https://rekvizitai.vz.lt/en/company/skuodo_bartuvos_vidurine_mokykla/
https://rekvizitai.vz.lt/en/company/vilniaus_mindaugo_vidurine_mokykla/
https://rekvizitai.vz.lt/en/company/lietuvos_akluju_ir_silpnaregiu_ugdymo_centras/
https://rekvizitai.vz.lt/en/company/mazeikiu_senamiescio_pagrindine_mokykla/
https://rekvizitai.vz.lt/en/company/panevezio_rozyno_progimnazija/
https://rekvizitai.vz.lt/en/company/siauliu_suaugusiuju_vidurine_mokykla/
https://rekvizitai.

https://rekvizitai.vz.lt/en/company/suportus/
https://rekvizitai.vz.lt/en/company/talentynas/
https://rekvizitai.vz.lt/en/company/tauneta/
https://rekvizitai.vz.lt/en/company/tekfina/
https://rekvizitai.vz.lt/en/company/topdotas_it/
https://rekvizitai.vz.lt/en/company/transperale/
https://rekvizitai.vz.lt/en/company/vienas_ragas/
https://rekvizitai.vz.lt/en/company/viviteka/
https://rekvizitai.vz.lt/en/company/wise_guys_pre_seed_management/
https://rekvizitai.vz.lt/en/company/aplicas/
https://rekvizitai.vz.lt/en/company/ateities_inovacijos/
https://rekvizitai.vz.lt/en/company/bit_by_bit_developers/
https://rekvizitai.vz.lt/en/company/enkomas_ii/
https://rekvizitai.vz.lt/en/company/uab_internovus/
https://rekvizitai.vz.lt/en/company/it_projektas/
https://rekvizitai.vz.lt/en/company/jivos_laboratorija/
https://rekvizitai.vz.lt/en/company/kalbos_technologijos/
https://rekvizitai.vz.lt/en/company/a_kolesin_ii/
https://rekvizitai.vz.lt/en/company/kibernetines_nindzes/
https://rekvizitai.vz.

https://rekvizitai.vz.lt/en/company/i_daraskos_ii/
https://rekvizitai.vz.lt/en/company/i_milasiaus_tub/
https://rekvizitai.vz.lt/en/company/idea_nova_uab/
https://rekvizitai.vz.lt/en/company/idv_sprendimai/
https://rekvizitai.vz.lt/en/company/iguana_projects/
https://rekvizitai.vz.lt/en/company/mb_ilgaausis/
https://rekvizitai.vz.lt/en/company/uab_imax_lab/
https://rekvizitai.vz.lt/en/company/uab_incus_ir_partneriai/
https://rekvizitai.vz.lt/en/company/uab_infobeta/
https://rekvizitai.vz.lt/en/company/infokaukas/
https://rekvizitai.vz.lt/en/company/informaciniu_sistemu_konsultacijos/
https://rekvizitai.vz.lt/en/company/inmedija/
https://rekvizitai.vz.lt/en/company/inovatyviu_technologiju_sklaidos_centras/
https://rekvizitai.vz.lt/en/company/inovatus/
https://rekvizitai.vz.lt/en/company/uab_interactive_virtual_worlds/
https://rekvizitai.vz.lt/en/company/invest_in_me/
https://rekvizitai.vz.lt/en/company/agronur/
https://rekvizitai.vz.lt/en/company/it_consultum/
https://rekvizitai.vz.lt/e

https://rekvizitai.vz.lt/en/company/blue_oceans_pr/
https://rekvizitai.vz.lt/en/company/energijos_partneris/
https://rekvizitai.vz.lt/en/company/uab_fincon/
https://rekvizitai.vz.lt/en/company/globalgen_lithuania/
https://rekvizitai.vz.lt/en/company/intelektualiu_resursu_sistemu_konsultantai/
https://rekvizitai.vz.lt/en/company/lisodea/
https://rekvizitai.vz.lt/en/company/mobilusis_naikinimas/
https://rekvizitai.vz.lt/en/company/fortconsult/
https://rekvizitai.vz.lt/en/company/outsourcing_business_intelligence_consulting/
https://rekvizitai.vz.lt/en/company/vetdeza/
https://rekvizitai.vz.lt/en/company/privacy_partners/
https://rekvizitai.vz.lt/en/company/pro_rata/
https://rekvizitai.vz.lt/en/company/skills_access/
https://rekvizitai.vz.lt/en/company/sprendimu_garantas/
https://rekvizitai.vz.lt/en/company/svencioniu_verslo_informacinis_centras/
https://rekvizitai.vz.lt/en/company/uab_teeko_consulting/
https://rekvizitai.vz.lt/en/company/viesoji_istaiga_marijampoles_regiono_mokslo_ir_tec

https://rekvizitai.vz.lt/en/company/expertus_consultus/
https://rekvizitai.vz.lt/en/company/fabaltics/
https://rekvizitai.vz.lt/en/company/farmanis/
https://rekvizitai.vz.lt/en/company/uab_farmneta/
https://rekvizitai.vz.lt/en/company/feras/
https://rekvizitai.vz.lt/en/company/uab_fibre_cement_systems/
https://rekvizitai.vz.lt/en/company/fidum/
https://rekvizitai.vz.lt/en/company/finadtech/
https://rekvizitai.vz.lt/en/company/financial_investment_trust_uab/
https://rekvizitai.vz.lt/en/company/finansista/
https://rekvizitai.vz.lt/en/company/finansu_avilys/
https://rekvizitai.vz.lt/en/company/uab_finansugidas_lt/
https://rekvizitai.vz.lt/en/company/finbalt_invest/
https://rekvizitai.vz.lt/en/company/finergija/
https://rekvizitai.vz.lt/en/company/finjustus/
https://rekvizitai.vz.lt/en/company/finlab_partners/
https://rekvizitai.vz.lt/en/company/risklita/
https://rekvizitai.vz.lt/en/company/finmarkas/
https://rekvizitai.vz.lt/en/company/finpro_lt/
https://rekvizitai.vz.lt/en/company/fintim

...

https://rekvizitai.vz.lt/en/company/agrosaule_1/
https://rekvizitai.vz.lt/en/company/agutis/
https://rekvizitai.vz.lt/en/company/ajeka/
https://rekvizitai.vz.lt/en/company/alkos_ekologija/
https://rekvizitai.vz.lt/en/company/alkos_energija/
https://rekvizitai.vz.lt/en/company/alkos_sviesa/
https://rekvizitai.vz.lt/en/company/alminta/
https://rekvizitai.vz.lt/en/company/uab_alnaja/
https://rekvizitai.vz.lt/en/company/alungu/
https://rekvizitai.vz.lt/en/company/amrisana/
https://rekvizitai.vz.lt/en/company/uab_amzina_energija/
https://rekvizitai.vz.lt/en/company/alternatyvi_saules_energija/
https://rekvizitai.vz.lt/en/company/uab_apricis/
https://rekvizitai.vz.lt/en/company/arda_group/
https://rekvizitai.vz.lt/en/company/arksolis/
https://rekvizitai.vz.lt/en/company/astroinvesticijos_pl/
https://rekvizitai.vz.lt/en/company/uab_atsakinga_energija/
https://rekvizitai.vz.lt/en/company/atsinaujinancios_energetikos_fondas/
https://rekvizitai.vz.lt/en/company/uab_atsinaujinantieji_istekliai/
h

https://rekvizitai.vz.lt/en/company/uab_rgb_baltic/
https://rekvizitai.vz.lt/en/company/zenono_skeiverio_imone/
https://rekvizitai.vz.lt/en/company/legimajus/
https://rekvizitai.vz.lt/en/company/elektroviga/
https://rekvizitai.vz.lt/en/company/grinera/
https://rekvizitai.vz.lt/en/company/inzinerijos_spektras/
https://rekvizitai.vz.lt/en/company/ismanus_inzineriniai_sprendimai/
https://rekvizitai.vz.lt/en/company/keliu_inzinerines_sistemos/
https://rekvizitai.vz.lt/en/company/komfortera/
https://rekvizitai.vz.lt/en/company/nightjar_finance_co_atstovybe/
https://rekvizitai.vz.lt/en/company/uab_omega_ventus/
https://rekvizitai.vz.lt/en/company/ravintuva/
https://rekvizitai.vz.lt/en/company/telsiu_inzinerines_sistemos/
https://rekvizitai.vz.lt/en/company/visi_inzineriniai_sprendimai/
https://rekvizitai.vz.lt/en/company/zemaitijos_ranga/
https://rekvizitai.vz.lt/en/company/stabena/
https://rekvizitai.vz.lt/en/company/greidaila/
https://rekvizitai.vz.lt/en/company/infinity_sav_lietuva/
https

https://rekvizitai.vz.lt/en/company/akimirka_automobiliu_sporto_klubo_imone/
https://rekvizitai.vz.lt/en/company/akmenes_autoklubas/
https://rekvizitai.vz.lt/en/company/alsengiris_kauno_medziotoju_klubas/
https://rekvizitai.vz.lt/en/company/arnetu_namas_kedainiu_jaunimo_menu_centras/
https://rekvizitai.vz.lt/en/company/atgaiva_papiles_seniunijos_pagyvenusiu_zmoniu_klubas/
https://rekvizitai.vz.lt/en/company/choras_bel_canto/
https://rekvizitai.vz.lt/en/company/en_den_du_siauliu_moksleiviu_namu_muzikos_klubas_studija/
https://rekvizitai.vz.lt/en/company/ignalinos_rajono_kulturos_centras_duksto_filialas/
https://rekvizitai.vz.lt/en/company/jaunasis_jureivis_klaipedos_vaiku_ir_jaunimo_klubas/
https://rekvizitai.vz.lt/en/company/jo_jo_pramogos_vaikams/
https://rekvizitai.vz.lt/en/company/jotainiu_bendruomenes_namai/
https://rekvizitai.vz.lt/en/company/julijos_jociutes_ii/
https://rekvizitai.vz.lt/en/company/jungtines_pramogos/
https://rekvizitai.vz.lt/en/company/kamertonas_kauno_misrus_cho

https://rekvizitai.vz.lt/en/company/litcapital_1/
https://rekvizitai.vz.lt/en/company/uab_lsv_intergroup/
https://rekvizitai.vz.lt/en/company/uab_simpleta/
https://rekvizitai.vz.lt/en/company/city_plaza/
https://rekvizitai.vz.lt/en/company/drusvalas/
https://rekvizitai.vz.lt/en/company/finansu_bite/
https://rekvizitai.vz.lt/en/company/finansu_bite_verslui/
https://rekvizitai.vz.lt/en/company/uab_sauel/
https://rekvizitai.vz.lt/en/company/vilnius_factoring_company/
https://rekvizitai.vz.lt/en/company/satola/
https://rekvizitai.vz.lt/en/company/verslo_konsultantai/
https://rekvizitai.vz.lt/en/company/bendras_finansavimas/
https://rekvizitai.vz.lt/en/company/enterge/
https://rekvizitai.vz.lt/en/company/finolita_unio/
https://rekvizitai.vz.lt/en/company/litcapital_asset_management/
https://rekvizitai.vz.lt/en/company/paskolu_klubas/
https://rekvizitai.vz.lt/en/company/nextury_ventures/
https://rekvizitai.vz.lt/en/company/unicredit_leasing/
https://rekvizitai.vz.lt/en/company/tbf_finance/
h

https://rekvizitai.vz.lt/en/company/lietuvos_medziotoju_ir_zveju_draugijos_silales_skyriaus_juros_medziotoju_burelis/
https://rekvizitai.vz.lt/en/company/lietuvos_medziotoju_ir_zveju_draugijos_silales_skyriaus_kvedarnos_medziotoju_klubas/
https://rekvizitai.vz.lt/en/company/lietuvos_medziotoju_ir_zveju_draugijos_traku_skyrius/
https://rekvizitai.vz.lt/en/company/lietuvos_mzd_kaisiadoriu_skyriaus_miskininkas_medziotoju_burelis/
https://rekvizitai.vz.lt/en/company/linksmasis_bebras/
https://rekvizitai.vz.lt/en/company/linksmuciu_medziotoju_klubas/
https://rekvizitai.vz.lt/en/company/luokes_medziotoju_burelis/
https://rekvizitai.vz.lt/en/company/magnum_hunt/
https://rekvizitai.vz.lt/en/company/marijampoles_medziotoju_klubas_zaltytis/
https://rekvizitai.vz.lt/en/company/mazeikiu_rajono_medziotoju_klubas_vetra/
https://rekvizitai.vz.lt/en/company/medziotoju_burelis_dubysa/
https://rekvizitai.vz.lt/en/company/medziotoju_burelis_lepsyne/
https://rekvizitai.vz.lt/en/company/medziotoju_burelis_

In [85]:
i = ['https://rekvizitai.vz.lt/en/company/kingsbury_baltic_properties/']
scrape(i)

57 Have been found.
Trying proxy: 185.128.136.231:3128
Succes! Using proxy: 185.128.136.231:3128
Succesfully scraped URL: https://rekvizitai.vz.lt/en/company/kingsbury_baltic_properties/
Scraping took: 10.65609647097881 seconds using proxy: 185.128.136.231:3128


In [77]:
print(soup)

NameError: name 'soup' is not defined

type(links[0])